In [1]:
import os
from pathlib import Path
from typing import Optional

import fastmri
import h5py
import matplotlib.pyplot as plt
import numpy as np
import torch
from data_utils import *
from datasets import *
from fastmri.data.transforms import tensor_to_complex_np
from skimage.metrics import peak_signal_noise_ratio, structural_similarity
from torch.utils.data import DataLoader, TensorDataset

from model import *
from torch.optim import SGD, Adam, AdamW
from train_utils import *

In [16]:
from pathlib import Path
path_to_data = '/itet-stor/mcrespo/bmicdatasets-originals/Originals/fastMRI/brain/multicoil_train/'
n_volumes = 1

dataset = KCoordDataset(path_to_data=path_to_data, n_volumes=4, n_slices=2 ,with_mask=False, center_frac=0.15)
dataloader = DataLoader(
        dataset,
        batch_size=120_000,
        num_workers=0, # This is needed to make processing faster 
        shuffle=False,
        # sampler=DistributedSampler(dataset),
        pin_memory=True,
    )

In [17]:
#####################################################################
# volume embedding
#####################################################################
embeddings_vol = torch.nn.Embedding(
    len(dataset.metadata), 256
)
torch.nn.init.normal_(
    embeddings_vol.weight.data, 0.0, 0.001
)

coil_size = []
for i in range(len(dataset.metadata)):
    _, n_coils, _, _ = dataset.metadata[i]["shape"]
    coil_size.append(n_coils)
########################################################
total_n_coils = torch.cumsum(torch.tensor(coil_size), dim=0)[-1]

start_indx = torch.tensor([0] + list(torch.cumsum(torch.tensor(coil_size), dim=0)[:-1]))

embeddings_coil = torch.nn.Embedding(total_n_coils.item(), 64)
torch.nn.init.normal_(
    embeddings_coil.weight.data, 0.0, 0.01
)

print(embeddings_coil)

Embedding(76, 64)


In [ ]:


for batch_idx, (inputs, targets) in enumerate(dataloader):
    # inputs, targets = inputs.to(self.device), targets.to(self.device)
    
    coords = inputs[:, 1:]
    vol_ids = inputs[:, 0].long()
    coil_id = inputs[:,-1].long()
    
    ## Map the coil_id to the embedding_coil_id table
    idx_start_coil = start_indx[vol_ids]
    coil_index_table = idx_start_coil + coil_id
    
    if batch_idx == 0:
        break
    
    # col_ids = inputs[:, -1].long()
    # shape = dataloader.dataset.metadata[vol_ids]["shape"]
    # latent_vol = embeddings_vol(vol_ids)
    # latent_coil = embeddings_coil[vol_ids](col_ids.unsqueeze(0))
    

In [ ]:
latent_coil = embeddings_coil(coil_index_table)

In [28]:
print(latent_coil[0])
print(latent_coil[1])

tensor([ 7.1697e-04,  1.6054e-02, -8.1535e-03, -4.3804e-03, -1.4798e-03,
         1.0080e-02,  1.5267e-03, -7.8560e-03, -1.4763e-02, -6.3897e-03,
         2.2738e-03, -4.8656e-03, -1.0216e-02,  2.7503e-02,  1.4525e-02,
         1.9802e-03, -5.4871e-03,  6.7677e-03,  1.6262e-02,  2.9267e-03,
         1.8971e-02, -1.1292e-02,  5.8535e-03, -8.5662e-03,  4.1489e-03,
        -6.2677e-03,  9.8695e-03,  9.1102e-03,  1.2742e-02, -2.8866e-02,
         1.7633e-03, -1.4269e-02, -2.2633e-03, -1.1309e-02, -2.3321e-03,
        -3.9699e-03,  1.6685e-02, -1.3623e-02,  1.0326e-03,  1.3341e-02,
        -1.6296e-02,  1.0357e-02,  2.2920e-03,  8.5684e-03, -1.2156e-02,
        -4.3018e-03,  2.7980e-02,  5.8493e-03,  1.1699e-02, -1.1763e-02,
         1.5692e-03, -1.0518e-02,  1.2219e-02,  2.8622e-03,  3.3609e-04,
        -1.2874e-02,  2.4047e-02,  4.8773e-05,  1.3925e-03, -3.4696e-03,
        -6.3902e-03,  5.2315e-03,  2.9739e-03,  1.3944e-02],
       grad_fn=<SelectBackward0>)
tensor([-0.0072, -0.0039,  0.

In [ ]:
dataloader.dataset.metadata[

TypeError: unhashable type: 'numpy.ndarray'

In [10]:
import torch
import torch.nn as nn

# Define the number of coils (Nc) for each volume and the embedding size
num_coils_per_volume = [20, 20, 16]  # Replace with actual Nc values
embedding_dim = 64

# Create a list of nn.Embedding for each volume
embeddings = nn.ModuleList([
    nn.Embedding(num_coils, embedding_dim) for num_coils in num_coils_per_volume
])

# Example input from dataloader
# input_coors = [vol_ID, kx, ky, kz, coilID] (batch_size, 5)
input_coors = torch.tensor([[0, 0.1, 0.2, 0.3, 5],  # vol_ID = 0, coilID = 5
                            [1, 0.5, 0.6, 0.7, 12],  # vol_ID = 1, coilID = 12
                            [2, 0.8, 0.9, 1.0, 7]])  # vol_ID = 2, coilID = 7

# Extract vol_IDs and coil_IDs from input_coors

# Fetch embeddings for each input coordinate
batch_embeddings = []
for coors in enumerate(input_coors):
    vol_ids = coors[:, 0].long()  # Batch of volume IDs
    coil_ids = coors[:, -1].long()  # Batch of coil IDs

    embedding_vector = embeddings[vol_ids](coil_ids.unsqueeze(0))  # Lookup for vol_id and coil_id
    batch_embeddings.append(embedding_vector)

# Combine embeddings into a batch tensor
batch_embeddings = torch.cat(batch_embeddings, dim=0)
print(batch_embeddings.shape)  # (batch_size, embedding_dim)


TypeError: tuple indices must be integers or slices, not tuple

In [24]:
inputs[:,0].long()

tensor([0, 0, 0,  ..., 0, 0, 0])